In [51]:
import segmentation_models as sm
import numpy as np
import sys, os
from tensorflow.keras.utils import normalize
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

In [52]:
sys.path.insert(1, 'E:/Research/course research projects/data-driven software engineering/gui-element-detection/src')
from constants import VINS_MERGED_YOLO_SPLITS_IMAGES as imgs_path
from constants import VINS_MERGED_YOLO_SPLITS_LABELS as labels_path
from util.data_utils import LazyDataLoader, Splits
# from ..util.data_utils import LazyDataLoader, Splits
# from ..constants import VINS_MERGED_YOLO_SPLITS_IMAGES as imgs_path
# from ..constants import VINS_MERGED_YOLO_SPLITS_LABELS as labels_path

In [53]:
batch_size = 12
classes = 12 + 1

In [54]:
def get_data_loader(split):
    return LazyDataLoader(split=split, images_path=imgs_path, labels_path=labels_path, batch_size=batch_size,
                          n_classes=classes, resize=True, shape=(416, 416), normalize=True)

In [55]:
train_loader = get_data_loader(Splits.TRAIN)
validation_loader = get_data_loader(Splits.VALIDATION)
test_loader = get_data_loader(Splits.TEST)

In [56]:


activation = 'softmax'
learning_rate = 0.0001
opt = Adam(learning_rate)
# dice_loss = sm.losses.DiceLoss()  # makes sense as the end result is highly imbalanced
# class_weights=tf.convert_to_tensor(np.array([0.25, 0.25, 0.25, 0.25]))
# focal_loss = sm.losses.CategoricalFocalLoss()  # why not Categorical CE ?
# total_loss = dice_loss + (focal_loss * 1)
total_loss = sm.losses.CategoricalCELoss()
metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.IOUScore(threshold=0.9,name='iou score @ 0.9'), sm.metrics.FScore(threshold=0.5),
           sm.metrics.Precision(), sm.metrics.Recall()]  # this is all they have :|

In [57]:
RESNET_BACKBONE = 'resnet34'
preprocess_inputs = sm.get_preprocessing(RESNET_BACKBONE)

train_loader = preprocess_inputs(train_loader)
test_loader = preprocess_inputs(test_loader)
validation_loader = preprocess_inputs(validation_loader)

In [58]:
model1 = sm.Unet(RESNET_BACKBONE, encoder_weights='imagenet', classes=13, activation=activation)
model1.compile(opt, total_loss, metrics=metrics)

In [59]:
model1.summary()

Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 data (InputLayer)              [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 bn_data (BatchNormalization)   (None, None, None,   9           ['data[0][0]']                   
                                3)                                                                
                                                                                                  
 zero_padding2d_170 (ZeroPaddin  (None, None, None,   0          ['bn_data[0][0]']                
 g2D)                           3)                                                         

In [ ]:
model1.fit(train_loader,
                      epochs=100,
                      verbose=1,
                      validation_data=validation_loader)

(12, 416, 416, 3)
(12, 416, 416, 13)
Epoch 1/100
(12, 416, 416, 3)
(12, 416, 416, 13)
(12, 416, 416, 3)
(12, 416, 416, 13)
  1/303 [..............................] - ETA: 52:42 - loss: 0.2667 - iou_score: 0.1540 - iou score @ 0.9: 0.1538 - f1-score: 0.1542 - precision: 0.0771 - recall: 0.2900(12, 416, 416, 3)
(12, 416, 416, 13)
  2/303 [..............................] - ETA: 2:25 - loss: 0.2652 - iou_score: 0.1155 - iou score @ 0.9: 0.1923 - f1-score: 0.1157 - precision: 0.0772 - recall: 0.3626 (12, 416, 416, 3)
(12, 416, 416, 13)
  3/303 [..............................] - ETA: 2:26 - loss: 0.2622 - iou_score: 0.1028 - iou score @ 0.9: 0.1538 - f1-score: 0.1029 - precision: 0.0772 - recall: 0.3161(12, 416, 416, 3)
(12, 416, 416, 13)
  4/303 [..............................] - ETA: 2:24 - loss: 0.2597 - iou_score: 0.0964 - iou score @ 0.9: 0.1347 - f1-score: 0.0966 - precision: 0.0772 - recall: 0.2756(12, 416, 416, 3)
(12, 416, 416, 13)
  5/303 [..............................] - ETA: 2:2

In [ ]:
model1.save('res34bb_UNET_100epochs.hdf5')